In [1]:
%matplotlib inline

from imageio import imread
import numpy as np
from skimage import color, filters, img_as_float
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def get_tile(tile_row, tile_col, img, img_gray, img_binary):
    tile_h = int(img.shape[0]/5)
    tile_w = int(img.shape[1]/5)
    y0, y1, x0, x1 = tile_row*tile_h, ((tile_row+1)*tile_h), tile_col*tile_w, (tile_col+1)*tile_w
    tile = img[y0:y1, x0:x1, :]
    tile_gray = img_gray[y0:y1, x0:x1]
    tile_binary = img_binary[y0:y1, x0:x1]
    return tile, tile_gray, tile_binary

In [76]:
from scipy.misc import imresize
MAX_SIZE = 225


def im_normalized_read(fname, size_limit = MAX_SIZE):
    from skimage.transform import resize
    img =  imread(fname)
    max_len = np.max(img.shape)
    if max_len <= size_limit:
        return img_as_float(img)
    ratio = size_limit/max_len
    new_img = imresize(img, size=ratio, interp='bicubic')
    return img_as_float(new_img)

# Color Moments

In [5]:
def calc_moment3(arr, moment1):
    cubed = np.mean((arr-moment1)**3)
    return math.pow(abs(cubed),1/3) * (1,-1)[cubed<0]

In [6]:
def calc_moments(arr):
    moment1 = np.mean(arr)
    moment2 = np.mean((arr-moment1)**2)**(1/2)
    moment3 = calc_moment3(arr, moment1)
    return moment1, moment2, moment3

In [7]:
def calc_4_rgb_moments(rgbarr):
    r1, r2, r3 = calc_moments(rgbarr[:, :, 0])
    g1, g2, g3 = calc_moments(rgbarr[:, :, 1])
    b1, b2, b3 = calc_moments(rgbarr[:, :, 2])
    gray = color.rgb2gray(rgbarr)
    gr1, gr2, gr3 = calc_moments(gray)
    return [r1, r2, r3, g1, g2, g3, b1, b2, b3, gr1, gr2, gr3]

# DOOG filter


Based on "Preattentive texture discrimination with early vision mechanisms", Fig. 2.

In [8]:
def G(y0, sigmax, sigmay, x, y):
    # The definition of G seems wrong, I refer original Yang's paper.
    # return (1/(2*math.pi* sigmax*sigmay)) *np.exp(- (x**2) + (y- y0/sigmay)**2)
    dimx = len(x)
    dimy = len(y)
    return (1/(2*math.pi* sigmax*sigmay)) *np.exp(- (1/2) * (((x/sigmax)**2).reshape(1, dimx) + (((y- y0)/sigmay)**2).reshape(dimy, 1)))

In [9]:
def DOOG2(x, y, sigma, r=3):
    sigmay = sigma
    sigmax = r*sigma
    ya = sigma
    yc = -sigma
    return -G(ya, sigmax, sigmay, x, y) + 2*G(0, sigmax, sigmay, x, y) - G(yc, sigmax, sigmay, x, y)

In [10]:
def DOOG2_weight(sigma, r=3, truncate=4):
    # similar filter size logic as filters.gaussian of scipy.
    lw = int(truncate * sigma + 0.5)
    return DOOG2(np.arange(-lw, lw+1), np.arange(-lw, lw+1), sigma, r)

In [11]:
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.convolve.html
from scipy.ndimage.filters import convolve

In [12]:
def kernel2feature(img, weights):
    return np.mean([np.mean(convolve(img[:, :, i], weights)) for i in range(3)])

In [13]:
# use global variable for speed
# use sigma of 1, 2, 4, 8

KERNELS = [DOOG2_weight(i) for i in [1, 2, 4, 8]]

In [14]:
def tile2doog_features(tile):
    # to make scale similar
    return [1000*kernel2feature(tile, kernel) for kernel in KERNELS]

# Orientation Filter

In [15]:
from scipy.stats import norm

In [16]:
def DOG(y, sigma1=0.5, sigma2=1.5):
    fy1 = norm.pdf(y, scale=sigma1)
    fy2 = norm.pdf(y, scale=sigma2)
    dog = fy1 - fy2
    return dog

In [17]:
def calc_f1(x, y, sigma):
    fx = norm.pdf(x, scale=sigma).reshape(1, len(x))
    fy = DOG(y).reshape(len(y), 1)
    # should access f1[y][x]
    return fx*fy

In [18]:
from scipy.signal import hilbert

In [19]:
def calc_f1f2(x, y, sigma):
    f1 = calc_f1(x, y, sigma)
    f2 = np.imag(hilbert(f1))
    return f1, f2

In [20]:
def rotation_indices(theta, shape):
    """return ([y1, y2, y3, ...], [x1, x2, x3, ...]). Use for numpy indices"""
    y_lim, x_lim = shape
    cos = math.cos(theta)
    sin = math.sin(theta)
    pairs = [(min(y_lim-1, max(0, int(x*sin + y*cos))),
              min(x_lim-1, max(0, int(x*cos - y*sin))))
            for y in range(y_lim)
            for x in range(x_lim)]
    return list(zip(*pairs))
    

### calc weights

In [21]:
def orientation_weights(theta, sigma, truncate=4):
    # similar filter size logic as filters.gaussian of scipy.
    lw = int(truncate * sigma + 0.5)
    
    x = np.arange(-lw, lw+1)
    y = np.arange(-lw, lw+1)
    f1, f2 = calc_f1f2(x, y, sigma)
    inds = rotation_indices(theta, f1.shape)
    f1r = f1[inds].reshape(f1.shape)
    f2r = f2[inds].reshape(f2.shape)
    return f1r, f2r

In [22]:
def orientation_energy(img, theta, sigma=1.5, truncate=4):
    f1, f2 = orientation_weights(theta, sigma)
    ex2 = convolve(img, f1)**2
    ey2 = convolve(img, f2)**2
    # make scale similar
    return 1000*np.mean(ex2+ey2)

In [23]:
def orientation_12_energy(gray_img):
    return [orientation_energy(gray_img, i*(2*math.pi)/12) for i in range(12)]

# Shape features

In [24]:
from skimage.filters import threshold_otsu

In [25]:
def gray2binary(gray_img):
    threds = threshold_otsu(gray_img)
    return gray_img <= threds

In [26]:
import skimage

### perimeter, area

In [27]:
# Paper says the ratio of the area to the perimeter squared, but inverse is much more common, which is compactness.
# standard definition divide this value with 4pi, but it's not important for our case.

def compactness(img_binary):
    l = skimage.measure.perimeter(img_binary)
    area = img_binary.sum()
    if area == 0:
        return 1, area
    return (l**2)/area, area

### Moment of inertia

In [28]:
def calc_moment_of_inertia(gray_img):
    # cr, cc = ``M[1, 0] / M[0, 0]``, ``M[0, 1] / M[0, 0]`
    M1 = skimage.measure.moments(gray_img, order=1)
    cr = M1[1, 0]/M1[0, 0]
    cc = M1[0, 1]/M1[0, 0]
    mu2 = skimage.measure.moments_central(gray_img, cr, cc, order=2)
    # paper says shape feature is 4. So I use mu2_20 and mu2_02 as a separate features
    # return (mu2[2, 0]+mu2[0, 2])/mu2[0, 0]
    M0sq = mu2[0, 0]**2
    return mu2[2, 0]/M0sq, mu2[0, 2]/M0sq

In [29]:
from skimage.morphology import convex_hull_image
def ratio_convex_hull(img_bainary, area):
    hull_area = convex_hull_image(img_bainary).sum()
    return area/hull_area

### Combine all shape features

In [30]:
def shape_features(img_gray, img_binary):
    comp, area = compactness(img_binary)
    mom = calc_moment_of_inertia(img_gray)
    if(img_binary.any()):
        ratio3 = ratio_convex_hull(img_binary, area)
    else:
        ratio3 = 1
    return [comp, mom[0], mom[1], ratio3]

## Combine all features

In [31]:
def tile2features(tile, tile_gray, tile_binary):
    rgb_moments12 = calc_4_rgb_moments(tile)
    doog4 = tile2doog_features(tile)
    orientation12 = orientation_12_energy(tile_gray)
    shape4 = shape_features(tile_gray, tile_binary)
    return [elem for arr in [rgb_moments12, doog4, orientation12, shape4]
            for elem in arr]

# img2features

In [32]:
def get_tile_features(tile_row, tile_col, img, img_gray, img_binary):
    tile, tile_gray, tile_binary = get_tile(tile_row, tile_col, img, img_gray, img_binary)
    return tile2features(tile, tile_gray, tile_binary)

In [33]:
def img2features(img):
    img_gray = color.rgb2gray(img)
    img_binary = gray2binary(img_gray)
    featuresList = [get_tile_features(row, col, img, img_gray, img_binary) for row in range(5)
                    for col in range(5)]
    return [f for fs in featuresList
               for f in fs]

# Create features and save to dataframe

In [82]:
from models.modelutils import dir2filedict

Using TensorFlow backend.


In [83]:
fdict = dir2filedict("data")

In [84]:
files = sorted(f for fs in fdict.values() for f in fs)

In [85]:
len(files)

11815

In [86]:
sfiles = files[0:100]

### Below here is too slow.

In [ ]:
current = ""

In [45]:
def files2features(files):
    global current
    res = []
    for f in files:
        current = f
        res.append(img2features(img_as_float(imread(f))))
    return res

In [88]:
%%time
flist = list(map(lambda file: img2features(im_normalized_read(file)), sfiles[0:2]))
# flist = files2features(sfiles)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


CPU times: user 19.2 s, sys: 2.39 s, total: 21.5 s
Wall time: 21.6 s


In [91]:
len(flist[0])

800

In [ ]:
len(flist)

In [48]:
current

'data/bay/0000.jpeg'

In [73]:
files[0:5]

['data/bay/0000.jpeg',
 'data/bay/0001.jpeg',
 'data/bay/0002.jpeg',
 'data/bay/0003.jpeg',
 'data/bay/0004.jpeg']

In [74]:
files[1000:1005]

['data/beach/0149.jpeg',
 'data/beach/0150.jpeg',
 'data/beach/0152.jpeg',
 'data/beach/0153.jpeg',
 'data/beach/0154.jpeg']

In [75]:
files[2000:2005]

['data/birds/0283.jpeg',
 'data/birds/0285.jpeg',
 'data/birds/0286.jpeg',
 'data/birds/0287.jpeg',
 'data/birds/0288.jpeg']

# Size evaluation


Using original size takes too much time due to large image size.
I evaluate to scale down to 1024, 640, 224, and found that most of the feature is quite similar.

The only exception of scale invariant traits is perimeter (compactness).
Theoretically, perimeter must scale the same ratio to image scale ratio, but the perimeter decrease much drastic than image scale.
This is because many bubles are merged while scale down image, that cause perimeter to reduce non-continuously.

But from the purpose of our feature extraction (predict concept), I guess these difference is not essential.
So use 224 for computational time.


In [35]:
# target = "data/bay/0001.jpeg"
target = 'data/bay/0000.jpeg'

In [36]:
def load_three(file, size=10000):
    img = im_normalized_read(file, size)
    img_gray = color.rgb2gray(img)
    img_binary = gray2binary(img_gray)
    return img, img_gray, img_binary

In [41]:
img, img_gray, img_binary = load_three(target)

In [42]:
img2, img2_gray, img2_binary = load_three(target, 1024)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()


In [43]:
img3, img3_gray, img3_binary = load_three(target, 640)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()


In [44]:
img4, img4_gray, img4_binary = load_three(target, 224)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()


In [54]:
%%time
org = get_tile_features(3, 3, img, img_gray, img_binary)

CPU times: user 11.5 s, sys: 228 ms, total: 11.7 s
Wall time: 11.9 s


In [55]:
%%time
f2 = get_tile_features(3, 3, img2, img2_gray, img2_binary)

CPU times: user 1.69 s, sys: 200 ms, total: 1.89 s
Wall time: 1.9 s


In [56]:
%%time
f3 = get_tile_features(3, 3, img3, img3_gray, img3_binary)

CPU times: user 1.22 s, sys: 140 ms, total: 1.36 s
Wall time: 1.37 s


In [57]:
%%time
f4 = get_tile_features(3, 3, img4, img4_gray, img4_binary)

CPU times: user 396 ms, sys: 56 ms, total: 452 ms
Wall time: 458 ms


In [79]:
np.array(org)-np.array(small)

array([  2.84158281e-03,   5.40832774e-03,   6.74004865e-03,
         4.89992021e-05,   1.35740563e-02,  -2.23009775e-02,
        -1.97152761e-03,   1.14532318e-02,  -1.99309068e-02,
         4.96743236e-04,   1.23676295e-02,  -2.73996822e-02,
         7.03990450e-05,   2.55697027e-04,   4.22333031e-04,
         5.26492168e-04,   1.29258513e+02,   1.10846336e+03,
         8.57062861e+02,   2.18777501e+00,   8.70882524e+00,
         2.50402808e+01,   3.83836342e-03,   2.44782002e-02,
         1.48670882e-01,   1.72181216e-01,   1.92997536e+03,
         5.38305047e+02,   3.06119728e+03,   1.64125071e-03,
         6.61167269e-04,  -3.34041820e-02])

In [59]:
import pandas as pd

In [61]:
pd.DataFrame({"f2":f2, "f3":f3, "f4":f4, "org":org})

,f2,f3,f4,org
0,0.344816,0.338217,0.281489,0.347657
1,0.257947,0.247008,0.209387,0.263355
2,0.230126,0.219276,0.208969,0.236866
3,0.657734,0.657691,0.654379,0.657783
4,0.112543,0.101710,0.072685,0.126117
5,-0.107601,-0.094233,-0.064984,-0.129902
6,0.644449,0.644653,0.667825,0.642477
7,0.186532,0.181078,0.148367,0.197986
8,-0.191789,-0.185444,-0.168567,-0.211720
9,0.590281,0.588862,0.576110,0.590778


### Why perimeter (feature 28) is so different?

In [62]:
skimage.measure.perimeter(img4_binary)

2037.965691583343

In [63]:
skimage.measure.perimeter(img_binary)/skimage.measure.perimeter(img4_binary)

194.92025197814365

In [67]:
skimage.measure.perimeter(img_binary)/skimage.measure.perimeter(img3_binary)

31.837669734658764

In [68]:
skimage.measure.perimeter(img_binary)/skimage.measure.perimeter(img2_binary)

12.247876436358119

In [69]:
max(img.shape)/224, max(img.shape)/640, max(img.shape)/1024

(17.285714285714285, 6.05, 3.78125)

In [70]:
17.28**2, 6.05**2, 3.78125**2

(298.5984, 36.6025, 14.2978515625)

In [79]:
skimage.measure.perimeter(img3_binary)/skimage.measure.perimeter(img4_binary)

6.1223152825770972

In [80]:
640/224

2.857142857142857

In [71]:
%%time
img2features(img)

CPU times: user 4min 45s, sys: 4.14 s, total: 4min 49s
Wall time: 4min 50s


[0.81944670877639414,
 0.056294294654326012,
 0.03630273951454282,
 0.96319416935012003,
 0.026315287370640636,
 -0.02091375815531246,
 0.99897283447215024,
 0.0022668845130980652,
 -0.0029097831647827634,
 0.9352274757335024,
 0.030212558298285858,
 -0.017994633598105315,
 0.21307002943196363,
 0.77389363976045233,
 1.278234831351694,
 1.5934832891383994,
 0.18167539203745636,
 6.5962162333412451,
 5.3121150452457027,
 0.0038510991554546407,
 0.035242919944245582,
 0.16189115058835027,
 2.4996231040496457e-05,
 0.00015907192708681256,
 0.00096574552457223564,
 0.0011182147223108463,
 12.40423586649972,
 2.1045178702363216,
 1,
 0.17380893778186202,
 0.10046360561847434,
 1,
 0.80883002408310245,
 0.055726573058388254,
 0.0401948569416258,
 0.95997017146127994,
 0.026442378500790363,
 -0.017802429699052092,
 0.99922288909977008,
 0.0019953871368174106,
 -0.002734311341335585,
 0.93068301108515206,
 0.030332970826031418,
 -0.012405933470702632,
 0.21202899505499775,
 0.77011248909718799

In [72]:
%%time
img2features(img2)

CPU times: user 42.7 s, sys: 4.16 s, total: 46.9 s
Wall time: 46.9 s


[0.81952491602050892,
 0.05633977282941955,
 0.0363459265438004,
 0.96319579499181962,
 0.02631098912543841,
 -0.020928555685463944,
 0.99897765317662779,
 0.0022291617431949825,
 -0.0028445330317562077,
 0.93524560518554067,
 0.030226046067988753,
 -0.018003152521381655,
 0.21307651369566738,
 0.77391719131508652,
 1.2782737312933936,
 1.5935317828936357,
 0.18185648439203125,
 6.6166691425660167,
 5.3392456471310199,
 0.003850695468158707,
 0.035590234145286587,
 0.16269216547091025,
 2.4996965370413304e-05,
 0.00015874410618581968,
 0.00096305216991863455,
 0.0011174158131639284,
 12.379038151676651,
 2.1023479644071998,
 1,
 0.17260376960720247,
 0.099922748217115775,
 1,
 0.8089413953645006,
 0.055784795759608645,
 0.04025603491189164,
 0.96004175822325499,
 0.026407809947625201,
 -0.017793882260044882,
 0.99922489104415213,
 0.0019542016793428316,
 -0.002659487999043412,
 0.93075803499215615,
 0.030327091458234014,
 -0.012389645794380254,
 0.21204316288493613,
 0.7701639482043555

In [73]:
%%time
img2features(img3)

CPU times: user 28.9 s, sys: 4.1 s, total: 33 s
Wall time: 33.4 s


[0.81876874279123446,
 0.055799123005003011,
 0.03606937605845081,
 0.96303813437139552,
 0.026274511126590048,
 -0.02075483743373405,
 0.9990300605536333,
 0.0021523435873624271,
 -0.0027738947827210324,
 0.93497590653835061,
 0.03010447729298053,
 -0.01772625789535562,
 0.21301052891745806,
 0.77367752738715978,
 1.2778778800229378,
 1.593038303629118,
 0.18198302760914661,
 6.6290403415291888,
 5.3571611324354595,
 0.0038483396083530464,
 0.03584487195284207,
 0.16323413835664705,
 2.4982004968671032e-05,
 0.00015838491228796842,
 0.00096031018446596326,
 0.0011160862457943875,
 12.350886329687988,
 2.0993840771334531,
 1,
 0.1765886854858609,
 0.10173305738373135,
 1,
 0.80811743079584797,
 0.055183689896909402,
 0.0398874155195125,
 0.95977256343713957,
 0.02639563923598421,
 -0.017589461915091056,
 0.99926002018454452,
 0.0019020950955679081,
 -0.0026114677290480035,
 0.93039289338235298,
 0.030209336255905422,
 -0.011931397527824884,
 0.2119621183718555,
 0.76986958567304187,
 1

In [74]:
%%time
img2features(img4)

CPU times: user 9.95 s, sys: 1.35 s, total: 11.3 s
Wall time: 11.3 s


[0.81673735324850927,
 0.054298281801669693,
 0.035124759492176305,
 0.96246235171184469,
 0.026237035882305252,
 -0.020276724217614804,
 0.99912717438072418,
 0.0019353877091763691,
 -0.0025045845685772567,
 0.93413932325281201,
 0.029813868387105848,
 -0.01693685717628016,
 0.21281825987253203,
 0.77297918519709596,
 1.2767244330042662,
 1.5916003843172404,
 0.18296532320788497,
 6.707893197083119,
 5.435674269817115,
 0.0038419650335373666,
 0.036976780108245587,
 0.16574109687509053,
 2.4933355775313042e-05,
 0.00015690014382957471,
 0.00094867663625943345,
 0.0011108258379007206,
 12.228906481992807,
 2.0919484929433252,
 1,
 0.17941079766345774,
 0.10305337484065051,
 1,
 0.8062972524432972,
 0.05357568388707664,
 0.038924057393843886,
 0.95951195525232047,
 0.026121322686096005,
 -0.01678629013566275,
 0.99927162087405486,
 0.0018532840324756376,
 -0.0025383266799542336,
 0.92982050279673001,
 0.029710682228399513,
 -0.01022289803982617,
 0.21180361991194768,
 0.7692939019395750

# Try and Error